## A dropdown for sales by country

As you were working away at your desk, the global sales manager for the global e-commerce company visited with a new data viz challenge. She noted that the recent sales graphs you produced were interesting, but she wanted to be able to filter and focus on only one country at a time and see the category aggregation data update.

You gladly accept the challenge, comforted that your expertise in Dash callbacks will be just the thing to breathe some interactive life into your Dash app.

The manager noted that it is important to use the corporately approved, abbreviated country names in the dropdown:

UK = United Kingdom, GM = Germany, FR = France, AUS = Australia, HK = Hong Kong.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link,style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Country Select'),
        # Add a dropdown with identifier
        dcc.Dropdown(id='country_dd',
        # Set the available options with noted labels and values
        options=[
            {'label':'UK', 'value':'United Kingdom'},
            {'label':'GM', 'value':'Germany'},
            {'label':'FR', 'value':'France'},
            {'label':'AUS', 'value':'Australia'},
            {'label':'HK', 'value':'Hong Kong'}],
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            # Add a graph component with identifier
            dcc.Graph(id='major_cat'),
            html.H2('Major Category', 
            style={ 'border':'2px solid black', 'width':'200px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ])], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    # Set the input and output of the callback to link the dropdown to the graph
    Output(component_id='major_cat', component_property='figure'),
    Input(component_id='country_dd', component_property='value')
)

def update_plot(input_country):
    country_filter = 'All Countries'
    sales = ecom_sales.copy(deep=True)
    if input_country:
        country_filter = input_country
        sales = sales[sales['Country'] == country_filter]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(
        title=f'Sales in {country_filter}', data_frame=ecom_bar_major_cat, x='Total Sales ($)', y='Major Category', color='Major Category',
                 color_discrete_map={'Clothes':'blue','Kitchen':'red','Garden':'green','Household':'yellow'})
    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## Fixing a broken dashboard

Oh no! While transferring your interactive sales dashboard to production, the file was corrupted. Most of the file was able to be recovered; however, there are some crucial functions and elements that have gaps, largely around the triggered callback function.

Can you use your knowledge of Dash callbacks to fix this dashboard?

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link,style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Country Select'),
        dcc.Dropdown(id='country_dd',
        options=[
            {'label':'UK', 'value':'United Kingdom'},
            {'label':'GM', 'value':'Germany'},
            {'label':'FR', 'value':'France'},
            {'label':'AUS', 'value':'Australia'},
            {'label':'HK', 'value':'Hong Kong'}],
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            dcc.Graph(id='major_cat'),
            html.H2('Major Category', 
            style={ 'border':'2px solid black', 'width':'200px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ])], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )
@app.callback(
    Output(component_id='major_cat', component_property='figure'),
    Input(component_id='country_dd', component_property='value')
)

def update_plot(input_country):
    # Set a default value
    country_filter = 'All Countries'
    # Ensure the DataFrame is not overwritten
    sales = ecom_sales.copy(deep=True)
    # Create a conditional to filter the DataFrame if the input exists
    if input_country:
        country_filter = input_country
        sales = sales[sales['Country'] == country_filter]
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(
        title=f'Sales in {country_filter}', data_frame=ecom_bar_major_cat, x='Total Sales ($)', y='Major Category', color='Major Category',
                 color_discrete_map={'Clothes':'blue','Kitchen':'red','Garden':'green','Household':'yellow'})
   	# Return the figure
    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## Date picker for sales data

The interactive sales dashboard you created was a smash hit; however, it is vital for the sales team to be able to monitor daily sales to analyze patterns and trends. You have been asked if you can extend your work to include a filter to view sales by major category on a particular date?

This sounds like a job for a date picker! You are definitely able to assist with the brief.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, date
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Sale Date Select'),
        # Create a single date picker with identifier
        dcc.DatePickerSingle(id="sale_date",
            # Set the min/max dates allowed as the min/max dates in the DataFrame
            min_date_allowed=ecom_sales['InvoiceDate'].min(),
            max_date_allowed=ecom_sales['InvoiceDate'].max(),
            # Set the initial visible date
            date=date(2011, 4, 11),
            initial_visible_month=date(2011,4,11),
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
      		# Add a component to render a Plotly figure with the specified id
            dcc.Graph(id="sales_cat"),
            html.H2('Daily Sales by Major Category', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ]),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    Output(component_id='sales_cat', component_property='figure'),
    Input(component_id='sale_date', component_property='date')
)
def update_plot(input_date):
    
    sales = ecom_sales.copy(deep=True)
    if input_date:
        sales = sales[sales['InvoiceDate'] == input_date]
        
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(
        title=f'Sales on: {input_date}',data_frame=ecom_bar_major_cat, orientation='h', 
        x='Total Sales ($)', y='Major Category')

    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## A date picker callback

Oh no! It has happened again. The date picker dashboard you created has become corrupted, and the callback code is unreadable.

Can you recall how to integrate a date picker component with a callback to get the dashboard back on track?

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, date
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Sale Date Select'),
        dcc.DatePickerSingle(id='sale_date',
            min_date_allowed=ecom_sales['InvoiceDate'].min(),
            max_date_allowed=ecom_sales['InvoiceDate'].max(),
            initial_visible_month=date(2011,4,1),
            date=date(2011,4,11),
            style={'width':'200px', 'margin':'0 auto'})
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            dcc.Graph(id='sales_cat'),
            html.H2('Daily Sales by Major Category', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ]),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )
# Create a callback and link
@app.callback(
    Output(component_id='sales_cat', component_property='figure'),
    Input(component_id='sale_date', component_property='date')
)
def update_plot(input_date):
    # Ensure the DataFrame is not overwritten
    sales = ecom_sales.copy(deep=True)
    
	# Conditionally filter the DataFrame using the input
    if input_date:
        sales = sales[sales['InvoiceDate'] == input_date]
        
    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
    bar_fig_major_cat = px.bar(
        title=f'Sales on: {input_date}',data_frame=ecom_bar_major_cat, orientation='h', 
        x='Total Sales ($)', y='Major Category')
	
    # Return the figure to render
    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## Slider for sales data

Analysis of trends and patterns in the global e-commerce company's data has been greatly enhanced with your date pickers. There is a further request to be able to see what categories are driving the 'big ticket' orders.

The company has asked you to build a tool whereby they can select a value (any value, really!) and a graph will show how many sales had an order value greater than that value, split by major category.

You know this would be facilitated well by a range input.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link, 
        style={'margin':'30px 0px 0px 0px' }),
  html.H1('Sales breakdowns'),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls'),
        html.Br(),
        html.H3('Minimum OrderValue Select'),
        # Add a slider input
        dcc.Slider(id='value_slider',
            # Set the min and max of the slider
            min=ecom_sales["OrderValue"].min(),
            max=ecom_sales["OrderValue"].max(),
            # Start the slider at 0
            value = 0 ,
            # Increment the slider by 50 each notch
            step = 50,
            vertical=False)
        ],
        style={'width':'350px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            dcc.Graph(id='sales_cat'),
            html.H2('Sales by Major Category', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ]),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )


@app.callback(
    Output(component_id='sales_cat', component_property='figure'),
    Input(component_id='value_slider', component_property='value')
)
def update_plot(min_val):
    
    sales = ecom_sales.copy(deep=True)

    if min_val:
        sales = sales[sales['OrderValue'] >= min_val]

    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].size().reset_index(name='Total Sales Volume')

    bar_fig_major_cat = px.bar(
        title=f'Sales with order value: {min_val}',data_frame=ecom_bar_major_cat, orientation='h', 
        x='Total Sales Volume', y='Major Category')

    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## Re-using Dash components

Your dashboards are starting to get quite lengthy in terms of the number of lines of code. The good news is, they are getting more attention around the organization. One of the senior managers saw your recent date picker sales dashboard and loved it! However, they had a few notes.

They want the logo to comply with corporate styling and align with the external website. The CSS for this is given below. They also want to include this logo in a few more places; on either side of the control panel and down at the bottom.

To ensure your code doesn't get unmanageable, you think this would be a great opportunity to take advantage of a Python function to create a reusable component.

The corporate logo style is: 'margin':'20px 20px 5px 5px', 'border':'1px dashed lightblue', 'display':'inline-block'

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, date
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])

app = dash.Dash(__name__)

# Create a reusable component function called add_logo
def add_logo():
    # Add a component that will render an image
    corp_logo = html.Img(
        src=logo_link, 
        # Add the corporate styling
        style={'margin':'20px 20px 5px 5px',
        'border':'1px dashed lightblue',
        'display':'inline-block'})
    
    return corp_logo

app.layout = html.Div([
  # Insert the logo (1)
  add_logo(),
  html.Br(),
  html.H1('Sales breakdowns'),
  html.Br(),
  html.Br(),
  html.Div(
    children=[
    html.Div(
        children=[
        # Insert the logo (2)
        add_logo(),
        html.H2('Controls', style={'margin':'0 10px','display':'inline-block'}),
        # Insert the logo (3)
        add_logo(),
        html.H3('Sale Date Select'),
        html.Br(),
        dcc.DatePickerSingle(
            id='sale_date',
            min_date_allowed=ecom_sales.InvoiceDate.min(),
            max_date_allowed=ecom_sales.InvoiceDate.max(),
            initial_visible_month=date(2011,4,1),
            date=date(2011,4,11),
            style={'width':'200px', 'margin':'0 auto'}),
        ],
        style={'width':'550px', 'height':'350px', 'display':'inline-block', 'vertical-align':'top', 'border':'1px solid black', 'padding':'20px'}),
    html.Div(children=[
            dcc.Graph(id='sales_cat'),
            html.H2('Daily Sales by Major Category', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ]),
  	# Insert the logo (4)
    add_logo(),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    Output(component_id='sales_cat', component_property='figure'),
    Input(component_id='sale_date', component_property='date')
)
def update_plot(input_date):
    
    sales = ecom_sales.copy(deep=True)

    if input_date:
        sales = sales[sales['InvoiceDate'] == input_date]

    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')

    bar_fig_major_cat = px.bar(
        title=f'Sales on: {input_date}',data_frame=ecom_bar_major_cat, orientation='h', 
        x='Total Sales ($)', y='Major Category')

    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## DRY-styling Dash components

Your clever use of Python functions allowed you to save some space in the last exercise. Yet, you think there is more that can be done to refactor this dashboard.

An additional function has been created to render multiple HTML breaks, which you can utilize in the code. Along with this, there seems to be a lot of overlap in the CSS styling you have added to components. This can also be cleaned up, you think!

The common style elements you have noticed are: 'display':'inline-block','margin':'0 auto','padding':'20px'

The make_break() function has been defined for you on line 14, which takes an argument specifying how many HTML break tags to create and returns these in a list.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, date
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_sales['InvoiceDate'] = pd.to_datetime(ecom_sales['InvoiceDate'])

app = dash.Dash(__name__)

def make_break(num_breaks):
    br_list = [html.Br()] * num_breaks
    return br_list

def add_logo():
    corp_logo = html.Img(
        src=logo_link, 
        style={'margin':'20px 20px 5px 5px',
              'border':'1px dashed lightblue',
              'display':'inline-block'})
    return corp_logo

# Create a function to add corporate styling
def style_c():
    layout_style={'display':'inline-block',
    'margin':'0 auto',
    'padding':'20px'}
    return layout_style

app.layout = html.Div([
  add_logo(),
  # Insert two HTML breaks
  *make_break(2),
  html.H1('Sales breakdowns'),
  # Insert three HTML breaks
  *make_break(3),
  html.Div(
    children=[
    html.Div(
        children=[
        add_logo(),
        # Style using the styling function
        html.H2('Controls', style=style_c()),
        add_logo(),
        html.H3('Sale Date Select'),
        # Insert two HTML breaks
        *make_break(2),
        dcc.DatePickerSingle(
            id='sale_date',
            min_date_allowed=ecom_sales.InvoiceDate.min(),
            max_date_allowed=ecom_sales.InvoiceDate.max(),
            initial_visible_month=date(2011,4,1),
            date=date(2011,4,11),
          	# Add styling using the styling function
            style={'width':'200px'}.update(style_c())),
        ],
        style={'width':'550px', 'height':'350px', 'vertical-align':'top', 'border':'1px solid black',
        'display':'inline-block', 'margin':'0px 40px'}),
    html.Div(children=[
            dcc.Graph(id='sales_cat'),
            html.H2('Daily Sales by Major Category', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ]),
    add_logo(),
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    Output(component_id='sales_cat', component_property='figure'),
    Input(component_id='sale_date', component_property='date')
)
def update_plot(input_date):
    
    sales = ecom_sales.copy(deep=True)

    if input_date:
        sales = sales[sales['InvoiceDate'] == input_date]

    ecom_bar_major_cat = sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')

    bar_fig_major_cat = px.bar(
        title=f'Sales on: {input_date}',data_frame=ecom_bar_major_cat, orientation='h', 
        x='Total Sales ($)', y='Major Category')

    return bar_fig_major_cat


if __name__ == '__main__':
    app.run_server(debug=True)

## Searching product descriptions

The e-commerce company you have been working with has a new challenge for you.

They are interested in providing a way to explore their sales data by country but are encountering difficulties with the product Description column. Unfortunately, the product Description column is a text field with many different values, which won't look good in a dropdown. They may wish to explore colors, brands, or other key descriptors easily.

This sounds like a job for a user-input component!

Help the e-commerce company to enhance their Dash app by taking a text user input to filter the Description column of the provided ecom_sales dataset.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, date
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

def make_break(num_breaks):
    br_list = [html.Br()] * num_breaks
    return br_list

def add_logo():
    corp_logo = html.Img(
        src=logo_link, 
        style={'margin':'20px 20px 5px 5px',
              'border':'1px dashed lightblue',
              'display':'inline-block'})
    return corp_logo

def style_c():
    layout_style={
        'display':'inline-block',
        'margin':'0 auto',
        'padding':'20px',
    }
    return layout_style
  
app = dash.Dash(__name__)

app.layout = html.Div([
  add_logo(),
  *make_break(2),
  html.H1('Sales Dashboard'),
  *make_break(3),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls', style=style_c()),
        html.H3('Search Descriptions'),
        *make_break(2),
        # Add the required input
        dcc.Input(id='search_desc', type='text', 
        placeholder='Filter Product Descriptions',
        # Ensure input is triggered with 'Enter'
        debounce=True,
        # Ensure the plot can load without a selection
        required=False,
        style={'width':'200px', 'height':'30px'})
        ],
        style={'width':'350px', 'height':'350px', 'vertical-align':'top', 'border':'1px solid black',
        'display':'inline-block', 'margin':'0px 80px'}),
    html.Div(children=[
            dcc.Graph(id='sales_desc'),
            html.H2('Sales Quantity by Country', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ])
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    Output(component_id='sales_desc', component_property='figure'),
    Input(component_id='search_desc', component_property='value')
)
def update_plot(search_value):
    title_value = 'None Selected (Showing all)'

    sales = ecom_sales.copy(deep=True)

    # Undertake the filter here using the user input
    if search_value:
        sales = sales[sales['Description'].str.contains(search_value, case=False)]
        title_value = search_value

    fig = px.scatter(data_frame=sales, 
                    y='OrderValue', x='Quantity', color='Country',title=f'Sales with description text: {title_value}')

    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

##Analyzing top customer locations

From your good work with the e-commerce company, you have been asked to help them build on an existing report looking at their sales by country.

The sales manager is interested in understanding where their top customers are located. They don't currently have a clear idea of what 'top' means. Ideally, they can easily try a variety of different cut-off values to consider someone a top customer and see how the graph changes with different cut-off values.

Your task is to help the sales manager by using a range input to facilitate a live-update app that filters sales by a minimum OrderValue amount.

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import datetime, date
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'

def make_break(num_breaks):
    br_list = [html.Br()] * num_breaks
    return br_list

def add_logo():
    corp_logo = html.Img(
        src=logo_link, 
        style={'margin':'20px 20px 5px 5px',
              'border':'1px dashed lightblue',
              'display':'inline-block'})
    return corp_logo

def style_c():
    layout_style={
        'display':'inline-block',
        'margin':'0 auto',
        'padding':'20px',
    }
    return layout_style

app = dash.Dash(__name__)

app.layout = html.Div([
  add_logo(),
  *make_break(2),
  html.H1('Sales Dashboard'),
  *make_break(3),
  html.Div(
    children=[
    html.Div(
        children=[
        html.H2('Controls', style=style_c()),
        html.H3('Set minimum OrderValue'),
        *make_break(2),
        dcc.Input(
          # Create the specified input
          id='min_order_val', type='range', 
          min=50, max=550, value=50,
          # Ensure the callback is triggered only when the user stops moving the slider
          debounce=False,
        style={'width':'300px', 'height':'30px'})
        ],
        style={'width':'350px', 'height':'350px', 'vertical-align':'top', 'border':'1px solid black',
        'display':'inline-block', 'margin':'0px 80px'}),
    html.Div(children=[
            dcc.Graph(id='sales_country'),
            html.H2('Sales Quantity by Country', 
            style={ 'border':'2px solid black', 'width':'400px', 'margin':'0 auto'})
            ],
             style={'width':'700px','display':'inline-block'}
             ),
    ])
    ], 
  style={'text-align':'center', 'display':'inline-block', 'width':'100%'}
  )

@app.callback(
    Output(component_id='sales_country', component_property='figure'), 
    Input(component_id='min_order_val', component_property='value'))

def update_plot(input_val):
  
    if not input_val:
      input_val = 0
  
    sales = ecom_sales.copy(deep=True)

    # Check for input then use to subset data
    if input_val:
        input_val = round(float(input_val), 2)
        sales = sales[sales['OrderValue'] > input_val]
    
    fig = px.scatter(data_frame=sales, y='OrderValue', height=400,
                     x='Quantity', color='Country',
					 # Set the conditional title
                     title=f'Orders of Min Value ${input_val}')
    return fig
      
if __name__ == '__main__':
    app.run_server(debug=True)